## IMPLEMENTACIÓN K-MEANS

In [9]:
import numpy as np
from sklearn.cluster import KMeans as KMeans_sklearn

# Implementación personalizada de K-Means
class KMeansCustom:
    def __init__(self, n_clusters, max_iters=300):
        self.n_clusters = n_clusters
        self.max_iters = max_iters

    def fit(self, X):
        self.centroids = X[np.random.choice(X.shape[0], self.n_clusters, replace=False)]
        
        for _ in range(self.max_iters):
            distances = np.sqrt(((X - self.centroids[:, np.newaxis])**2).sum(axis=2))
            labels = np.argmin(distances, axis=0)
            new_centroids = np.array([X[labels == k].mean(axis=0) for k in range(self.n_clusters)])
            if np.all(self.centroids == new_centroids):
                break
            
            self.centroids = new_centroids
        
        self.labels_ = labels

# predict...
# make_blobs, para comparar kmeans evaluacionnn lo hace en el tutorial
"""Ver a ojo dibujando seis características noooo
Correlacion dibujando matriz y juntar(temperatura con color)
1) ESCALAR
2) PCA DOS O TRES NO VAMOS A PODER PINTAR MÁS.UTILIZANDO COMPONENTES PRINCIPALES.
3) PRUEBA CON DOS¿?QUE VARIABILIDAD, CERCA DE 80 OKEYYYY, SOLO DOS, SINO PRUEBA CON TRES
4) SI ES CON TRES QUE SE PUEDA MOVER CON EL RATON. EL CHIVA QUE DOSSSS Y PINTASSS.
5) JERARQUICO, KMEANS, DBSCAN, ETC.
6) 6-7 CLUSTERS, HAY QUE CARACTERIZARLAS Y GENERAR BOX-PLOTS PARA CADA CLASE Y VER COMO SE DIVIDEN
LAS CARACTERÍSTICAS
7)VER VALORES DE CIENTIFICOS Y CARACTERIZAR, SE PUEDE PROBAR ARBOL DE DECISIÓN"""
X = np.random.rand(100, 2)

# Número de clusters
n_clusters = 3

kmeans_custom = KMeansCustom(n_clusters=n_clusters)
kmeans_custom.fit(X)
labels_custom = kmeans_custom.labels_

# Usar la implementación de K-Means de scikit-learn
kmeans_sklearn = KMeans_sklearn(n_clusters=n_clusters)
labels_sklearn = kmeans_sklearn.fit_predict(X)

# Comparar los resultados
print("Etiquetas obtenidas con la implementación personalizada:")
print(labels_custom)
print("\nEtiquetas obtenidas con la implementación de scikit-learn:")
print(labels_sklearn)


Etiquetas obtenidas con la implementación personalizada:
[0 0 0 0 2 1 2 0 2 2 1 2 0 0 0 1 2 2 2 2 0 0 1 1 2 0 2 2 1 0 0 0 1 2 0 1 1
 1 1 1 0 0 0 2 1 2 0 1 0 0 1 2 1 0 0 1 2 0 2 0 0 0 1 1 2 2 2 0 0 2 2 0 2 0
 1 0 2 2 2 1 1 0 2 2 2 1 1 0 0 1 0 1 2 1 1 2 1 2 2 2]

Etiquetas obtenidas con la implementación de scikit-learn:
[1 1 2 2 2 0 0 2 0 0 0 0 1 1 1 0 2 2 0 0 1 2 0 0 0 1 0 2 0 2 1 2 0 0 2 0 0
 0 0 0 1 1 2 0 0 2 2 0 1 1 0 2 1 1 1 0 0 1 2 1 1 1 0 0 0 0 0 1 1 0 2 2 0 2
 0 1 0 2 0 0 0 2 0 2 0 0 0 1 1 0 2 1 0 0 0 2 0 0 0 2]
